%%

In [ ]:
import numpy as np
import keras_nlp
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_text as tf_text
from tensorflow import keras
from tensorflow.lite.python import interpreter
import time

%%

In [ ]:
gpt2_tokenizer = keras_nlp.models.GPT2Tokenizer.from_preset("gpt2_base_en")
gpt2_preprocessor = keras_nlp.models.GPT2CausalLMPreprocessor.from_preset(
    "gpt2_base_en",
    sequence_length=256,
    add_end_token=True,
)
gpt2_lm = keras_nlp.models.GPT2CausalLM.from_preset("gpt2_base_en", preprocessor=gpt2_preprocessor)

%%

In [ ]:
output = gpt2_lm.generate("My trip to Yosemite was", max_length=200)
print("\nGPT-2 output:")
print(output.numpy().decode("utf-8"))

%%

In [ ]:
output = gpt2_lm.generate("That Italian restaurant is", max_length=200)
print("\nGPT-2 output:")
print(output.numpy().decode("utf-8"))

%%

In [ ]:
cnn_ds = tfds.load('cnn_dailymail', as_supervised=True)

%

In [ ]:
from nltk import tokenize

import nltk
nltk.download('punkt')

for article, highlights in cnn_ds['train']:
  combination = article + tf.constant(' TL;DR ') + tf.strings.regex_replace(highlights, "\n", " ")
  word_count=len(tokenize.word_tokenize(str(combination.numpy())))
  if word_count < 256:
    print(combination.numpy())
    print(article.numpy())
    print(highlights.numpy())
    break

%%

In [ ]:
import progressbar

short_texts = []
total = len(cnn_ds['train'])
progressbar_update_freq = 1000
count = 0

widgets = [' [',
         progressbar.Timer(format= 'elapsed time: %s'),
         '] ',
           progressbar.Bar('*'),' (',
           progressbar.ETA(), ') ',
          ]
bar = progressbar.ProgressBar(
    maxval=total // progressbar_update_freq + 2,
    widgets=widgets).start()

In [ ]:
for article, highlights in cnn_ds['train']:
  combination = article + tf.constant(' TL;DR ') + tf.strings.regex_replace(highlights, "\n", " ")
  word_count = len(tokenize.word_tokenize(str(combination.numpy())))
  if word_count < 256:
    short_texts.append(combination)
  count += 1
  if count % progressbar_update_freq == 0:
    bar.update(count / progressbar_update_freq)

%%

In [ ]:
def save_texts(texts):
    np.savez('data/selected_texts.npz', texts)
    
def load_texts():
    restored_texts = list()
    with np.load('data/selected_texts.npz', allow_pickle=True) as data:
      for file in data.files:
        restored_texts.extend(data[file].tolist())
    return restored_texts

Save the list of short combinations of articles and summaries (sort of a checkpoint).

In [ ]:
save_texts(short_texts)

%%

In [ ]:
# short_texts = load_texts()

%%

In [ ]:
tf_train_ds = tf.data.Dataset.from_tensor_slices(short_texts)
processed_ds = tf_train_ds.map(gpt2_preprocessor, tf.data.AUTOTUNE).batch(20).cache().prefetch(tf.data.AUTOTUNE)
part_of_ds = processed_ds.take(100)

%%

In [ ]:
gpt2_lm.include_preprocessing = False

num_epochs = 1

lr = tf.keras.optimizers.schedules.PolynomialDecay(
    5e-5,
    decay_steps=part_of_ds.cardinality() * num_epochs,
    end_learning_rate=0.0,
)

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

gpt2_lm.compile(
    optimizer=keras.optimizers.experimental.Adam(lr),
    loss=loss,
    weighted_metrics=["accuracy"])

gpt2_lm.fit(part_of_ds, epochs=num_epochs)

%%